In [2]:
#import click
import pandas as pd
from sqlalchemy import create_engine
from tqdm.auto import tqdm

In [14]:
# Read a sample of the data
prefix = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/'
df_zones = pd.read_csv(prefix + 'taxi_zone_lookup.csv')

In [15]:
df_zones

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone
261,262,Manhattan,Yorkville East,Yellow Zone
262,263,Manhattan,Yorkville West,Yellow Zone
263,264,Unknown,NV,NaN


In [17]:
df_zones.dtypes

LocationID       int64
Borough         object
Zone            object
service_zone    object
dtype: object

In [3]:
chunksize=100000

In [47]:
# Read a sample of the data
prefix = 'https://d37ci6vzurychx.cloudfront.net/trip-data/'
df_parquet = pd.read_parquet(prefix + 'green_tripdata_2025-11.parquet')

In [51]:
df_parquet

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1.0,74,42,1.0,0.74,7.20,...,0.5,1.94,0.0,NaN,1.0,11.64,1.0,1.0,0.00,0.00
1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1.0,74,42,2.0,0.95,7.20,...,0.5,0.00,0.0,NaN,1.0,9.70,2.0,1.0,0.00,0.00
2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1.0,83,160,1.0,2.19,13.50,...,0.5,5.00,0.0,NaN,1.0,21.00,1.0,1.0,0.00,0.00
3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1.0,166,127,1.0,5.44,24.70,...,0.5,0.50,0.0,NaN,1.0,27.70,1.0,1.0,0.00,0.00
4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1.0,166,262,1.0,3.20,18.40,...,1.5,1.00,0.0,NaN,1.0,24.65,1.0,1.0,2.75,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46907,2,2025-11-30 19:58:34,2025-11-30 20:14:28,None,NaN,59,51,NaN,8.50,33.22,...,0.5,0.00,0.0,NaN,1.0,34.72,NaN,NaN,NaN,0.00
46908,2,2025-11-30 19:34:00,2025-11-30 19:46:00,None,NaN,74,151,NaN,1.73,13.86,...,0.5,0.77,0.0,NaN,1.0,16.13,NaN,NaN,NaN,0.00
46909,2,2025-11-30 21:46:46,2025-11-30 22:17:55,None,NaN,33,163,NaN,7.52,38.42,...,0.5,1.00,0.0,NaN,1.0,44.42,NaN,NaN,NaN,0.75
46910,2,2025-11-30 21:00:00,2025-11-30 21:15:00,None,NaN,16,95,NaN,5.61,24.67,...,0.5,0.00,0.0,NaN,1.0,26.17,NaN,NaN,NaN,0.00


In [9]:
df_parquet.columns

Index(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge',
       'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge',
       'cbd_congestion_fee'],
      dtype='object')

In [10]:
df_parquet.dtypes

VendorID                          int32
lpep_pickup_datetime     datetime64[us]
lpep_dropoff_datetime    datetime64[us]
store_and_fwd_flag               object
RatecodeID                      float64
PULocationID                      int32
DOLocationID                      int32
passenger_count                 float64
trip_distance                   float64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
ehail_fee                       float64
improvement_surcharge           float64
total_amount                    float64
payment_type                    float64
trip_type                       float64
congestion_surcharge            float64
cbd_congestion_fee              float64
dtype: object

In [7]:
df_chunks = [df_parquet[i:i+chunksize] for i in range(0, len(df_parquet), chunksize)]

In [17]:
type(df_chunks)

list

In [14]:
pg_user= "root"
pg_pass= "root"
pg_host="localhost"
pg_port=5432
pg_db= "ny_green_taxi"

In [15]:
engine = create_engine(f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}')

In [33]:
!uv add aiohttp requests

Resolved 117 packages in 185ms                                       
Prepared 7 packages in 77ms                                              
░░░░░░░░░░░░░░░░░░░░ [0/7] Installing wheels...                                 warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 7 packages in 16ms                                
 + aiohappyeyeballs==2.6.1
 + aiohttp==3.13.3
 + aiosignal==1.4.0
 + frozenlist==1.8.0
 + multidict==6.7.0
 + propcache==0.4.1
 + yarl==1.22.0


In [42]:
target_table="green_taxi_data"

In [46]:
import pyarrow.parquet as pq
import fsspec
from tqdm import tqdm

url = "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-11.parquet"

first = True

with fsspec.open(url, "rb") as f:
    pf = pq.ParquetFile(f)

    for batch in tqdm(pf.iter_batches(batch_size=chunksize)):
        df_chunk = batch.to_pandas()

        if first:
            # Create table schema only
            df_chunk.head(0).to_sql(
                name=target_table,
                con=engine,
                if_exists="replace",
                index=False
            )
            first = False

        # Append data
        df_chunk.to_sql(
            name=target_table,
            con=engine,
            if_exists="append",
            index=False,
            method="multi"   # MUCH faster inserts
        )


1it [00:14, 14.89s/it]
